<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#直接贪心法，sensor按照节点顺序连接client" data-toc-modified-id="直接贪心法，sensor按照节点顺序连接client-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>直接贪心法，sensor按照节点顺序连接client</a></span></li><li><span><a href="#最近邻贪心法，sensor连接最近的5个client" data-toc-modified-id="最近邻贪心法，sensor连接最近的5个client-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>最近邻贪心法，sensor连接最近的5个client</a></span></li><li><span><a href="#GA启发式" data-toc-modified-id="GA启发式-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>GA启发式</a></span></li></ul></div>

In [1]:
import numpy as np
import collections
import sys
import matplotlib.pyplot as plt
from tqdm import tqdm, trange
import random 
import copy
%matplotlib inline
from util import *



In [1]:
# 以下超参在util中定义了，这里只是为了方便复制
# NUM_OF_SENSORS = 100
# NUM_OF_CLIENTS = 100
# SERVERS_CONNECT_ABILITY = 5
# CLIENT_CONNECT_NEED = 3
# SENSOR_DISTANCE = 50
# LENGTH_OF_SQUARE = 400

# maxiter = 100
# SIZE = 10

# CROSSOVER_RATE = 0.5
# MUTATION_RATE = 0.001


'\nNUM_OF_SENSORS = 100\nNUM_OF_CLIENTS = 100\nSERVERS_CONNECT_ABILITY = 5\nCLIENT_CONNECT_NEED = 3\nSENSOR_DISTANCE = 50\nLENGTH_OF_SQUARE = 400\n\nmaxiter = 100\nSIZE = 10\n\nCROSSOVER_RATE = 0.5\nMUTATION_RATE = 0.001\n'

In [3]:
for i in range(10):
    seed = np.random.randint(0,1000)
    sensors, clients, distances = init_nodes_and_distance_matrix(seed)

    tmp = np.zeros_like(distances)
    tmp[distances<=SENSOR_DISTANCE] = 1
    tmp = np.sum(tmp, axis=0)
    max_client = np.sum(tmp>=CLIENT_CONNECT_NEED)

    _, num1 = greedy_most_near(sensors, clients, distances)
    _, num2,_ = GA(sensors, clients, distances)
    _, num3 = max_flow(sensors, clients, distances)

    print("理论：{}\t贪心：{}\t遗传：{}\t 最大流:{}".format(max_client, num1, num2, num3))
    


理论：75	贪心：68	遗传：73	 最大流:75
理论：81	贪心：72	遗传：77	 最大流:78
理论：85	贪心：76	遗传：82	 最大流:85
理论：91	贪心：78	遗传：87	 最大流:91
理论：81	贪心：75	遗传：79	 最大流:79
理论：77	贪心：67	遗传：71	 最大流:76
理论：68	贪心：59	遗传：67	 最大流:66
理论：79	贪心：71	遗传：75	 最大流:79
理论：74	贪心：64	遗传：69	 最大流:71
理论：83	贪心：70	遗传：78	 最大流:80
